In [19]:
import time
import MySQLdb
t1 = time.clock()
conn = MySQLdb.connect(host='localhost',user='root',passwd='9182736450',db='mozilla')
cur = conn.cursor()
cur.execute('SELECT * FROM bugs_activity WHERE fieldid=29 ORDER BY bug_id, bug_when;')
t2 = time.clock()
print t2-t1

108.195000419


In [20]:
t1 = time.clock()
sortedTran = cur.fetchall()
t2 = time.clock()
print t2-t1

0.210676423615


In [21]:
t1 = time.clock()
recDict = {}
problemList = []
tableDict = {}
lenDict = {}
tranDict = {}
curProblem = -1
tranStr = None
lastId = None
for item in sortedTran:
    if item[0] == curProblem:
        continue
    # encounter a new bug
    if item[0] not in recDict:
        # for last bug
        if tranStr:
            tableDict[lastId]={"transition":tranStr,"timeStamp":timeList}
        lastId = item[0]
        recDict[item[0]] = 1
        tranStr = item[4] + " " + item[5]
        timeList = [item[2],]
    # a problem bug
    elif item[4] != added:
        problemList.append(item[0])
        curProblem = item[0]
        tranStr = None
    # the following transition
    else:
        tranStr += " " + item[5]
        timeList.append(item[2])
    added = item[5]
t2 = time.clock()
print t2-t1

3.33666433462


In [22]:
len(tableDict)

694210

In [23]:
t1 = time.clock()
cur.execute('SELECT bug_id,creation_ts,bug_severity,priority,resolution,product_id FROM bugs')
t2 = time.clock()
print t2-t1

11.0058865555


In [24]:
t1 = time.clock()
bugsInfo = cur.fetchall()
t2 = time.clock()
print t2-t1

0.000154519231273


In [25]:
# append creation_ts and other info for a single bug
t1 = time.clock()
for bug in bugsInfo:
    if bug[0] in tableDict:
        tableDict[bug[0]]["timeStamp"].insert(0,bug[1])
        tableDict[bug[0]]["bug_severity"]=bug[2]
        tableDict[bug[0]]["priority"]=bug[3]
        tableDict[bug[0]]["resolution"]=bug[4]
        tableDict[bug[0]]["product_id"]=bug[5]
t2 = time.clock()
print t2-t1

2.07461454294


In [26]:
t1 = time.clock()
# create database
# sql = "CREATE DATABASE IF NOT EXISTS iwe DEFAULT CHARSET utf8 COLLATE utf8_general_ci;"
# cur.execute(sql)
# change database
cur.execute("USE iwe")
sql = """
CREATE TABLE `iwe_statusTran` (
  `pk` int unsigned AUTO_INCREMENT PRIMARY KEY,
  `bugId` int unsigned NOT NULL,
  `transition` varchar(80) NOT NULL DEFAULT '',
  `ts0` datetime NULL,
  `ts1` datetime NULL,
  `ts2` datetime NULL,
  `ts3` datetime NULL,
  `ts4` datetime NULL,
  `ts5` datetime NULL,
  `bug_severity` VARCHAR(64) NOT NULL,
  `priority` VARCHAR(64) NOT NULL,
  `resolution` VARCHAR(64) NOT NULL,
  `product_id` SMALLINT NOT NULL
);
"""
cur.execute(sql)
t2 = time.clock()
print t2-t1

0.191028533601


In [27]:
# insert into table
t1 = time.clock()
for bugId in tableDict:
    if len(tableDict[bugId]["timeStamp"]) > 6:
        continue
    sql = "INSERT INTO iwe_statusTran (bugId, transition"
    for i in range(len(tableDict[bugId]["timeStamp"])):
        sql += ", ts" + str(i)
    infoKeys = ("bug_severity","priority","resolution","product_id")
    for key in infoKeys:
        sql += ", " + key
    sql += ") VALUES(" + str(bugId) + ", '" + tableDict[bugId]["transition"] + "'"
    for ts in tableDict[bugId]["timeStamp"]:
        sql += ", '" + str(ts) + "'"
    for key in infoKeys:
        sql += ", '" + str(tableDict[bugId][key]) + "'"
    sql += ");"
    cur.execute(sql)
t2 = time.clock()
print t2-t1

93.8236753112


In [28]:
cur.close()
conn.commit()

In [9]:
cur.execute("SELECT * FROM iwe_statusTran WHERE bugId = 35")
cur.fetchall()

((1L,
  35L,
  'NEW RESOLVED VERIFIED',
  datetime.datetime(1998, 4, 7, 1, 37, 3),
  datetime.datetime(1998, 12, 12, 9, 6, 46),
  datetime.datetime(1999, 2, 26, 12, 55, 50),
  None,
  None,
  None,
  'minor',
  'P3',
  'WONTFIX',
  7),)

In [10]:
cur.execute('SELECT COUNT(*) FROM iwe_statusTran;')
cur.fetchall()

((680056L,),)

In [12]:
conn.commit()

In [38]:
t1 = time.clock()
sql = """CREATE INDEX statusTran_bugId 
ON iwe_statusTran(bugId);"""
cur.execute(sql)
t2 = time.clock()
print t2-t1

1.66608147088


In [45]:
t1 = time.clock()
sql = """CREATE INDEX statusTran_transition 
ON iwe_statusTran(transition);"""
cur.execute(sql)
t2 = time.clock()
print t2-t1

OperationalError: (1061, "Duplicate key name 'statusTran_transition'")

In [44]:
t1 = time.clock()
sql = """
SELECT * FROM iwe_statusTran WHERE transition = 'NEW RESOLVED VERIFIED'
"""
cur.execute(sql)
t2 = time.clock()
print t2-t1

3.19900742889


In [41]:
cur.fetchall()

((1L,
  35L,
  'NEW RESOLVED VERIFIED',
  datetime.datetime(1998, 4, 7, 1, 37, 3),
  datetime.datetime(1998, 12, 12, 9, 6, 46),
  datetime.datetime(1999, 2, 26, 12, 55, 50),
  None,
  None,
  None),
 (2L,
  36L,
  'NEW ASSIGNED RESOLVED VERIFIED',
  datetime.datetime(1998, 4, 7, 2, 4, 3),
  datetime.datetime(1998, 9, 4, 19, 7, 1),
  datetime.datetime(1999, 1, 20, 15, 0, 28),
  datetime.datetime(1999, 3, 22, 13, 42, 23),
  None,
  None),
 (3L,
  37L,
  'RESOLVED VERIFIED REOPENED RESOLVED VERIFIED',
  datetime.datetime(1998, 4, 7, 2, 20, 1),
  datetime.datetime(1999, 3, 4, 8, 54, 59),
  datetime.datetime(2000, 12, 25, 17, 50, 15),
  datetime.datetime(2000, 12, 25, 17, 52, 25),
  datetime.datetime(2000, 12, 25, 17, 53, 17),
  None),
 (4L,
  38L,
  'RESOLVED VERIFIED REOPENED RESOLVED VERIFIED',
  datetime.datetime(1998, 4, 7, 2, 30, 3),
  datetime.datetime(1999, 3, 4, 8, 55, 14),
  datetime.datetime(2000, 12, 25, 17, 50, 11),
  datetime.datetime(2000, 12, 25, 17, 51, 28),
  datetime.date

In [39]:
t1 = time.clock()
sql = """
SELECT bug_id,resolution FROM mozilla.bugs RIGHT JOIN 
iwe.iwe_statustran ON mozilla.bugs.bug_id = iwe.iwe_statustran.bugId"""
cur.execute(sql)
t2 = time.clock()
print t2-t1

6.91712875086


In [34]:
sss = cur.fetchall()

In [35]:
len(sss)

680056

In [37]:
sss[1]

(36, 'INVALID')